In [2]:
import pandas as pd
import numpy as np
import geopandas
import os
from datetime import date
pd.set_option('display.max_columns', None)

In [3]:
def str_to_path(path_str):
    '''Converts windows path string to os.path'''
    path_list = path_str.split('\\')
    path = os.path.join(path_list[0], os.sep, *path_list[1:])
    return path

In [11]:
def standardize_polygons(gdf, start_date, end_date, name, prop_type, unique_id, amount, units):
    
    gdf['start_date'] =  pd.to_datetime(gdf[start_date])
    gdf = gdf.loc[gdf.start_date >= '1-1-2020'].copy()
    gdf['end_date'] = pd.to_datetime(gdf[end_date])
    # Calculate area
    gdf = gdf.to_crs("EPSG:3310") # California Albers in meters
    gdf['amount'] = gdf[amount]
    gdf['units'] = units
    gdf['name'] = gdf[name]
    gdf['type'] = gdf[prop_type]
    gdf['unique_id'] = unique_id
    # Create x and y coordinates
    gdf['centroid'] = gdf.centroid
    gdf = gdf.set_geometry('centroid')
    gdf = gdf.to_crs("EPSG:4326")
    gdf['latitude'] = gdf['centroid'].y
    gdf['longitude'] = gdf['centroid'].x
    
    # Set crs back to CA Albers
    gdf = gdf.to_crs("EPSG:3310")
    gdf = gdf[['unique_id', 'name', 'type', 'start_date', 'end_date', 'amount', 'units', 'latitude', 'longitude', 'centroid']]
    
    return gdf


In [5]:
date = date.today().strftime("%Y%m%d")
#date = '20220331'
n_path = str_to_path(r"N:\Projects\Dashboard\data")
tnc_lands_fc = os.path.join(n_path, "TNC_Lands_{}.shp".format(date))
hr_path = str_to_path(r'N:\WATER_Program\projects\metrics\data')
hr_fc = os.path.join(hr_path, 'HR_benefit_points_20220830.shp')


table_dir = os.path.join(os.getcwd(), 'tables')
in_table = os.path.join(table_dir, 'CA_strategy_dashboard_metrics_v1.2_Apr_2022.xlsx')
out_table = os.path.join(table_dir, 'ca_strategy_points_{}.csv'.format(date))
out_shapefile = os.path.join(n_path, 'ca_strategy_points_{}.shp'.format(date))


In [6]:
df = pd.read_excel(in_table, sheet_name='CA_strategy_outcomes')
#df = df.replace(['nan'],np.NaN)
df = df.dropna(how='any', thresh=6).reset_index()
df['unique_id'] = df.program + "_" + df.strategy + "_" + df.outcome

In [7]:
df

,index,program,strategy,outcome,global outcome,2020 extent or rate in CA,2025 progress estimate,2025 progress estimate (%),2030 outcome target,units,math,notes,unique_id
0,0,Climate,sustainable resilient communities,avoided conversion,land avoided,"52,000 acres/yr",1.050000e+05,0.299709,3.503400e+05,acres,"BAU rate of conversion = 52,000 acres/yr, 2025...",320K avoided CV lands and farmland via greenpr...,Climate_sustainable resilient communities_avoi...
1,1,Climate,sustainable resilient communities,avoided emissions,climate avoided,"775,000 MTCO2e/yr",2.025000e+06,0.299981,6.750420e+06,MTCO2e,"775,000 MTCO2e/yr (7.75 MMT over 10 yrs), 2025...",soil and veg conservation via avoided conversi...,Climate_sustainable resilient communities_avoi...
2,3,Climate,wildlife corridors and crossings,crossing structures designed and funded,land improved,0,1.000000e+00,0.500000,2.000000e+00,crossing structures,NaN,was 3 crossing structures in October 2021 metr...,Climate_wildlife corridors and crossings_cross...
3,4,Climate,wildlife corridors and crossings,decreased mortality,land improved,NaN,1.000000e+00,0.333333,3.000000e+00,crossing structures,NaN,NaN,Climate_wildlife corridors and crossings_decre...
4,6,Climate,energy,avoided conversion,land avoided,"21,400 acres/yr",4.800000e+04,0.300000,1.600000e+05,acres,"2030 outcome = 21400 acres per year projected,...",assumes low impact siting = avoided impact on ...,Climate_energy_avoided conversion
5,7,Climate,energy,zero carbon energy (all CA),climate avoided,0.33,4.110000e+07,0.300000,1.370000e+08,MTCO2e,get to 50% by 2030 = 27% increase from BAU,note this is not TNC alone; climate emissions ...,Climate_energy_zero carbon energy (all CA)
6,9,Climate,nature based solutions,avoided emissions and/or increased sequestration,climate avoided,BAU,3.000000e+07,0.300000,1.000000e+08,MTCO2e,NaN,includes negative emissions; C neutrality cont...,Climate_nature based solutions_avoided emissio...
7,10,Climate,nature based solutions,investment in natural climate solutions,NaN,$X/yr,6.000000e+09,0.600000,1.000000e+10,$,$1Bn/yr * 10yrs,continuation of rough level in new budget,Climate_nature based solutions_investment in n...
8,11,Climate,nature based solutions,coastal wetlands restoration,land improved,NaN,1.030000e+04,0.297860,3.458000e+04,acres,NaN,NaN,Climate_nature based solutions_coastal wetland...
9,13,Climate,forests,planned & permitted,land improved,"2,400,000 acres",1.200000e+06,0.352941,3.400000e+06,acres,2025 = 35% of 2030 outcome,streamlining process,Climate_forests_planned & permitted


In [12]:
#Import TNC lands
gdf = geopandas.read_file(tnc_lands_fc)
#Fix names for public use
#TODO- move to pre-processing script
gdf['prop_name'] = np.where(gdf['PUB_NAME'].isna(), gdf['TRACTNAME'], gdf['PUB_NAME'])
gdf['prop_name'] = np.where(gdf['TNC_INTERE']=="TNC Easement", gdf['CONS_AREA'].astype(str) + ' Easement', gdf['prop_name'])
gdf['end_date'] = pd.to_datetime('1-1-2100')
gdf['area_acres'] = gdf.area / 4046.86



In [13]:
# Create point gdf for TNC lands strategies
#TODO - make this more standardized
    
gdf1 = standardize_polygons(gdf.loc[gdf.TNC_INTERE.isin(['TNC Easement', 'TNC Fee', 'TNC Historic'])].copy(),
                             start_date = 'PROT_DATE',
                             end_date = 'end_date',
                             name = 'prop_name',
                             prop_type = 'TNC_INTERE',
                             unique_id = 'Land_land protection_protection by TNC',
                            amount = 'area_acres',
                            units = 'acres'
                            
                            )
gdf2 = standardize_polygons(gdf.loc[gdf.TNC_INTERE.isin(['TNC Assist'])].copy(),
                             start_date = 'PROT_DATE',
                             end_date = 'end_date',
                             name = 'prop_name',
                             prop_type = 'TNC_INTERE',
                             unique_id = 'Land_land protection_protection by partners',
                            amount = 'area_acres',
                            units = 'acres'
                            )

gdf_all = pd.concat([gdf1, gdf2], ignore_index=True)

In [14]:
# Healthy Rivers
gdf_hr = geopandas.read_file(hr_fc)
gdf_hr['start_date'] = pd.to_datetime(gdf_hr.start_date).dt.tz_localize(None) # Fix localized datetime
gdf_hr.loc[gdf_hr['end_date'] == "ongoing", 'end_date'] = "1-1-2100" #change ongoing to 1/1/2100
gdf_hr1 = standardize_polygons(gdf_hr,
                             start_date = 'start_date',
                             end_date = 'end_date',
                             name = 'name',
                             prop_type = 'name',
                             unique_id = 'Water_healthy rivers_improved management',
                               amount = 'river_mi',
                               units = 'river_miles'
                            )
gdf_all = pd.concat([gdf_all, gdf_hr1], ignore_index=True)

In [15]:
# export data
gdf_shp = gdf_all.copy()
gdf_shp['start_date'] = gdf_shp['start_date'].dt.date.astype(str)
gdf_shp['end_date'] = gdf_shp['end_date'].dt.date.astype(str)
gdf_shp.to_file(out_shapefile)  

gdf_all.to_csv(out_table, encoding='utf-8')

d:\Users\kklausmeyer\AppData\Local\Continuum\anaconda3\envs\dashboard\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [16]:
gdf_shp.dtypes

unique_id       object
name            object
type            object
start_date      object
end_date        object
amount         float64
units           object
latitude       float64
longitude      float64
centroid      geometry
dtype: object